# 机器学习纳米学位
## 强化学习
## 项目 4: 训练智能出租车学会驾驶

## 实现基本的驾驶智能体
第一步，你的任务是让这个智能出租车在环境中动起来。此时，你不需要考虑任何行驶优化问题。注意这个智能车在每个路口都会获得下列信息。

 - 下一路径点位置（相对于当前位置和前进方向）；
 - 十字路口状态（红绿灯和是否有车）；
 - 当前最后期限值（剩余的时间步）；


要完成这个任务，要你的智能车在每个路口从下面这几个可能的动作中随机选择一个。 `(None, forward, left, right)` 忽略上面提到的输入信息。设置模拟环境中运行截止时间，即 `enforce_deadline` 设置为 `False`，并观察智能车的驾驶行为。

### 问题 1
*在你的报告中观察并记录智能车在采取随机动作时的行为。它最终到达目标位置了吗？还有什么其他有趣的现象值得记录下来？*

**回答:**

本题改动保存于agent_1_random.py中，改动点包括：28行，action改为random.choice(Environment.valid_actions)；44行，enforce_deadline设置为false，取消时间限制，确保可以观察尽量长的时间；48行，update_delay改为2s，方便观察每一步智能车的行动。

（说明：针对本项目中的4个主要问题，smartcab目录下有对应的四个agent代码类，分别是agent_1_random.py,agent_2_decide_state.py,agent_3_q_init.py,agent_4_parameter_optimized.py，对应每一题都有相应的改动）

智能车会在环境中“随机游走”，当取消了时间限制后，智能车总是能达到目标位置，但通常花费时间非常多。

由于环境大小有限，因此理论上只要有无限的时间，随机游走策略总能保证智能车达到目的地；但从几次观察结果来看，这个时间比预期的要长得多。

**（After Review）**

可以将enforce_deadline设置为True，update_delay设置为0，display设置为False，尝试运行多次agent_1_random.py，观察最后的统计值，可以的出随机游走的智能车的成绩：规定时间内到达平均20次，失败80次，平均交通违章次数770

## 训练智能车
现在你的智能车已经可以在环境中移动，你接下来的任务是确定一组合适的状态对智能出租车与它的环境建模。状态变量的主要来源是当前路口的当前输入（`inputs`），但并非所有都需要被表示出来。你可以显式地定义状态，或者用一些输入的组合作为一个隐式的状态。在每一时间节点中，你需要处理输入，用`self.state`更新智能车的当前状态。依然把 `enforce_deadline` 设置为 `False` 观察你的智能车在随着模拟的进行，如何汇报状态的改变。

### 问题 2
*你认为哪些状态适合对 **智能出租车** 和 **环境** 建模？为什么你认为在这个题目中这些状态是合适的？*

**回答:**

我认为适合场景建模的状态有：下一目标点next_waypoint,交通灯状态inputs['light']，路口对面车况inputs['oncoming']，路口左方车况inputs['left']

首先，下一目标点是必要的状态，没有这个变量，智能车就无法实时获取到达目标地点的指示。

其次，交通灯状态也是必须的，这个状态和路口车况共同决定了智能车采取的行动是否可以立即被执行。

最后，路口车况中，对面车况和左方车况也是需要考虑的因素：智能车可以左转时，如果路口对面有车直行，则需要等待对面直行后再左转；当交通灯为红灯且智能车决定右转时，如果左方有车直行，则智能车也需要让行。需要说明的是，路口右方的车况任何情况下都不会影响智能车的决策，因此不予考虑。

**（After Review）**

不考虑将deadline作为状态的最主要的原因是，deadline的状态太多了，如可选题目（下一题）的计算结果可知，选定的状态决定了智能车有384种动作-状态组合，如果再乘以deadline的所有可能，得到的总状态数会非常多，相应的学习矩阵也会非常大，矩阵完成收敛需要很长的训练时间。

### 可选题目
*在这个环境中，智能出租车总共可能有多少状态？这个数字足够让我们的智能车做 Q-Learning，使得它在每个状态可以做出基于训练的决策吗？如果是，为什么？如果不是，也说一下原因。*

**回答:**

根据问题2的结论，下一目标点的状态有3个（前，左，右），交通灯状态有两个（红，绿），对面车况以及左方车况各有4个（无车，有车直行，有车左转，有车右转），因此总共可能的状态是： 2×3×4×4=96 。如果再算上智能车本身可能采取的动作（停，直行，左转，右转），则总共有： 96×4=384种动作-状态组合。



## 实现 Q-Learning 智能车
当你的智能车能够理解输入信息，并且有一个环境状态的映射。你下一步的任务是为智能车实现 Q-Learning 算法，使得它可以在每一个时间节点，基于当前状态和动作的Q值做最佳动作选择。智能出租车所做的每个动作，都会产生一个基于环境状态的奖励。Q-Learning 智能车需要在更新Q值時，考虑这些奖励。实现完成，把模拟环境中的 enforce_deadline 设置为 True。运行模拟器，观察环境中的智能出租车在每一轮测试中的移动情况。

### 问题 3： 
*与一直选择随机动作相比，你发现智能车的行为有了什么样的变化？为什么会发生这种变化？*

**回答:**

在前1到2次尝试中，智能车的行为和之前随机游走模式的智能车的行为比较相似。但这以后，虽然智能车偶尔会有一些异常动作，但它的确开始可以做到越来越快的到达终点。

发生这种变化的原因在于采用了实现Q-Learning算法的决策方式，随着智能车越来越多次成功抵达目的地或超时失败，用于存储‘经验’的Q-Value-Dic就会持续更新收敛，智能车也就可以从Q-Value-Dic中获取到当前状态下应当采取的最‘合理’的动作。因此当Q-Value-Dic收敛后，智能车几乎总是可以采取最合理的动作序列到达目的地了。

为了避免智能车过于依赖以前的学习‘经验’（即每次都采取效用值最大的动作，有可能导致更好的选择一直无法探索到），代码中选择action的地方采用了epsilon-greedy方法，即epsilon概率采用随机动作的方式，剩余1-epsilon遵循‘经验’——这就是智能车偶尔会有一些异常动作的原因。

## 提高 Q-Learning 智能车
在这个项目中，你最终的任务是强化你的智能车，让它在经过足够的训练之后，能够在规定的时间内安全高效的抵达目的地。Q-Learning 算法中的参数，例如学习率（`alpha`），折扣因子（`gamma`）以及探索率（`epsilon`）这些都对智能车能否在每个状态学习出最佳动作选择有影响。调整他们，要提高你**智能出租车**的成功率：
 - 把测试次数，n_trials，在模拟器中设置成100。
 - 在 enforce_deadline 设置为 True 的情况下运行模拟器。（你还需要减小更新延迟 update_delay 并且把 display 参数设置为 False）。
 - 观察智能车的学习过程和成功率，特别是在后期的测试中的表现。
 - 调整上述参数中的一个或多个参数，迭代这个过程。
 
完成这个任务，你需要找到你认为对智能车能够成功学习的最佳参数组合。

### 问题 4
*把你实现基本Q-Learning时的参数调节过程记下来。哪个参数组合智能车表现最好？它最终的表现有多好？*

**回答:**

首先按照题目要求，将测试次数调整至100，更新延时调整为0，display设置为False。

按照grid-search参数搜索法，选定学习率(0.5,0.6,0.7),折扣率(0.2,0.3,0.4)，探索率（0.1,0.15,0.2），共27组实验,结果如下：

| alpha | gamma | epsilon | 成功率（10次实验平均） | 交通违章（10次实验平均） |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 0.5               |        0.2           |        0.10          |      96.9%      |    57.1       |
| 0.6               |        0.2           |        0.10          |      97.7%      |    51.0       |
| 0.7               |        0.2           |        0.10          |      97.2%      |    51.6       |
| 0.5               |        0.3           |        0.10          |      94.9%      |    55.9       |
| 0.6               |        0.3           |        0.10          |      96.2%      |    51.9       |
| 0.7               |        0.3           |        0.10          |      95.9%      |    53.8       |
| 0.5               |        0.4           |        0.10          |      92.2%      |    52.6       |
| 0.6               |        0.4           |        0.10          |      92.9%      |    51.0       |
| 0.7               |        0.4           |        0.10          |      93.2%      |    52.0       |
| 0.5               |        0.2           |        0.15          |      95.2%      |    75.4       |
| 0.6               |        0.2           |        0.15          |      94.5%      |    73.5       |
| 0.7               |        0.2           |        0.15          |      94.8%      |    75.6       |
| 0.5               |        0.3           |        0.15          |      90.3%      |    76.4       |
| 0.6               |        0.3           |        0.15          |      94.9%      |    75.3       |
| 0.7               |        0.3           |        0.15          |      92.4%      |    78.8       |
| 0.5               |        0.4           |        0.15          |      89.3%      |    76.7       |
| 0.6               |        0.4           |        0.15          |      90.4%      |    75.2       |
| 0.7               |        0.4           |        0.15          |      90.6%      |    77.4       |
| 0.5               |        0.2           |        0.2          |      90.5%      |    107.6       |
| 0.6               |        0.2           |        0.2          |      92.7%      |    100.7       |
| 0.7               |        0.2           |        0.2          |      92.4%      |    98.5       |
| 0.5               |        0.3           |        0.2          |      89.8%      |    105.5       |
| 0.6               |        0.3           |        0.2          |      90.0%      |    104.6       |
| 0.7               |        0.3           |        0.2          |      89.6%      |    101.3       |
| 0.5               |        0.4           |        0.2          |      85.8%      |    108.1       |
| 0.6               |        0.4           |        0.2          |      87.8%      |    101.4       |
| 0.7               |        0.4           |        0.2          |      86.9%      |    101.0       |


alpha是学习率，这个数值越大说明智能体会保留越少的之前的训练结果,更重视新尝试的经验。当alpha很大时，智能体每次只参考很小一部分(1-alpha)的先前训练结果，主要靠新尝试获取经验，那么Q矩阵的元素必然不容易收敛。反之，如果alpha过小，那么智能体对于新尝试的参考很小，绝大部分的经验来源于之前的经验，显然的，这样的学习率会导致Q矩阵以非常缓慢的速度向目标收敛，效果也不会很好。因此我选择了几个接近0.5的数值进行尝试：0.4，0.5，0.6，从下表（原始表中截取）可以看到，在gamma，epsilon固定的情况下，alpha为0.7,0.6时，通常比0.5时拥有较高的或持平的成功率，并且学习率0.6,0.7的成功率表现总体持平，考虑到alpha过大可能会影响到学习矩阵的最终收敛，因此选择0.6作为学习率参数。

| alpha | gamma | epsilon | 成功率（10次实验平均） | 交通违章（10次实验平均） |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 0.5               |        0.3           |        0.10          |      94.9%      |    55.9       |
| 0.6               |        0.3           |        0.10          |      96.2%      |    51.9       |
| 0.7               |        0.3           |        0.10          |      95.9%      |    53.8       |

| alpha | gamma | epsilon | 成功率（10次实验平均） | 交通违章（10次实验平均） |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 0.5               |        0.3           |        0.15          |      90.3%      |    76.4       |
| 0.6               |        0.3           |        0.15          |      94.9%      |    75.3       |
| 0.7               |        0.3           |        0.15          |      92.4%      |    78.8       |

| alpha | gamma | epsilon | 成功率（10次实验平均） | 交通违章（10次实验平均） |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 0.5               |        0.3           |        0.2          |      89.8%      |    105.5       |
| 0.6               |        0.3           |        0.2          |      90.0%      |    104.6       |
| 0.7               |        0.3           |        0.2          |      89.6%      |    101.3       |

gamma是折扣因子，这个值越大，智能体就会更重视记忆中的利益，越小，就会更重视眼前的利益。此处我的考虑是，通过记忆中的利益给出的最优动作综合考虑了智能体在到达目的地和避免交通违章之间的权衡，但对于智能体的每一步决策，更重要的应该是尽量不违反交通规则从而避免事故。出于这个考虑，我认为智能车在计算当前效用值时，应该适当“短视”一些，因此我选了3个很小的gamma值：0.2,0.3，0.4。从原始数据中我截取出了下表，可以看到，比起gamma为0.4的选择，0.2,0.3的gamma值成功率通常更高，且0.2的交通违章情况更少一点，因此此处我选择了0.2作为gamma值：

| alpha | gamma | epsilon | 成功率（10次实验平均） | 交通违章（10次实验平均） |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 0.6               |        0.2           |        0.10          |      97.7%      |    51.0       |
| 0.6               |        0.3           |        0.10          |      96.2%      |    51.9       |
| 0.6               |        0.4           |        0.10          |      92.9%      |    51.0       |

| alpha | gamma | epsilon | 成功率（10次实验平均） | 交通违章（10次实验平均） |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 0.6               |        0.2           |        0.15          |      94.5%      |    73.5       |
| 0.6               |        0.3           |        0.15          |      94.9%      |    75.3       |
| 0.6               |        0.4           |        0.15          |      90.4%      |    75.2       |

| alpha | gamma | epsilon | 成功率（10次实验平均） | 交通违章（10次实验平均） |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 0.6               |        0.2           |        0.2          |      92.7%      |    100.7       |
| 0.6               |        0.3           |        0.2          |      90.0%      |    104.6       |
| 0.6               |        0.4           |        0.2          |      87.8%      |    101.4       |

epsilon是探索率，为了避免智能体完全依赖训练结果，而设置的让智能体概率性的随机探索的机制。很明显的，随机动作越多，成功抵达目的地的不确定性也就越大，可以计算一下探索率分别为0.1,0.15,0.2时，上述实验中的成功率和事故率的平均值，如下表所示，很明显，探索率的增高会减小成功抵达的几率和交通违章率，我认为0.1的探索率就足够了。

| epsilon | 成功率 | 交通违章 |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
|        0.10          |      95.2%      |    52.9       |
|        0.15          |      92.5%      |    76.0       |
|        0.2          |      89.5%      |    103.2       |

综上，最终选择的参数组合是：alpha=0.6，gamma=0.2，epsilon=0.1，这种情况下成功率为97.7%，交通事故平均值为51.0.

### 问题 5
*你觉得你的智能车已经几乎找到了最佳策略吗？例如，能够在最短时间内到达目的地，不遇到任何惩罚。在这个问题中，你觉得应该怎样定义最佳策略？*

**回答:**

在当前的奖励、惩罚机制下，我认为智能车已经接近最佳策略了：97.7%以上的到达成功率和较低（当然平均51也不算很低）的交通违章率还算可以接受。如果希望降低交通违章次数，可以将environment.py的违反交通规则的惩罚提高（当前是-1.0）

本题中的最佳策略，我认为应该是：尽量不发生交通违章的前提下，在规定的时间内到达目的地。因为对于乘客而言，智能车的安全应该是最重要的，其次才是到达时间。相比较下，智能车能在规定时间内不发生任何事故安全抵达，比起不惜造成交通事故更快抵达目的地更合理一些。